In [1]:
import os
store_dir = '/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/'
og_evts = '/global/cfs/projectdirs/m3443/data/trackml-kaggle/train_10evts/'
os.environ['TRKXINPUTDIR']=f"{store_dir}shift_x/"
os.environ['TRKXOUTPUTDIR']= f"{store_dir}shift_x_pre/"

In [2]:
import pkg_resources
import yaml
import pprint
import random
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import tqdm
from os import listdir
from os.path import isfile, join
import matplotlib.cm as cm
import sys
import csv
import time
import pickle
import tqdm
from tqdm import tqdm
import tqdm.notebook as tq
from pathlib import Path
# %matplotlib widget

sys.path.append('/global/homes/c/caditi97/exatrkx-iml2020/exatrkx/src/')

# 3rd party
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from trackml.dataset import load_event
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint


# local import
from exatrkx import config_dict # for accessing predefined configuration files
from exatrkx import outdir_dict # for accessing predefined output directories
from exatrkx.src import utils_dir
from exatrkx.src import utils_robust
from utils_robust import *


# for preprocessing
from exatrkx import FeatureStore
from exatrkx.src import utils_torch

# for embedding
from exatrkx import LayerlessEmbedding
from exatrkx.src import utils_torch
from torch_cluster import radius_graph
from utils_torch import build_edges
from embedding.embedding_base import *

# for filtering
from exatrkx import VanillaFilter

# for GNN
import tensorflow as tf
from graph_nets import utils_tf
from exatrkx import SegmentClassifier
import sonnet as snt

# for labeling
from exatrkx.scripts.tracks_from_gnn import prepare as prepare_labeling
from exatrkx.scripts.tracks_from_gnn import clustering as dbscan_clustering

# track efficiency
from trackml.score import _analyze_tracks
from exatrkx.scripts.eval_reco_trkx import make_cmp_plot, pt_configs, eta_configs
from functools import partial

In [3]:
event_path = '/global/cfs/cdirs/m3443/data/trackml-kaggle/train_all/'
eventid = 'event000001005'

In [4]:
misl = [0,0.0025,0.005,0.0075,0.01,0.012,0.015,0.017,0.02,0.1,0.4,0.6,0.8,1]
evts = np.arange(1000,1050,1)

In [5]:
store_dir = '/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/volumes_shifted/'
og_evts = '/global/cfs/projectdirs/m3443/data/trackml-kaggle/train_10evts/'

In [6]:
def save_pickle(data, store_dir, store_path, eventid):
    with open('pathtofile','wb') as csvFile: 
        writer = csv.writer(csvFile)
        writer.writerows(csvstff)

In [7]:
def preprocess():
    action = 'build'
    config_file = pkg_resources.resource_filename("exatrkx",os.path.join('configs', config_dict[action]))
    with open(config_file) as f:
        b_config = yaml.load(f, Loader=yaml.FullLoader)
    pp = pprint.PrettyPrinter(indent=4)
    b_config['pt_min'] = 0
    b_config['endcaps'] = True
    b_config['n_workers'] = 2
    b_config['n_files'] = 10
    b_config['noise'] = 0
    pp.pprint(b_config)
    preprocess_dm = FeatureStore(b_config)
    preprocess_dm.prepare_data()

# Remove Volume 8 Layer 6

In [8]:
# def remove_v8l6(hits,cells,particles,truth,store_dir,ev):
    
#     nhits = hits[(hits['volume_id']==8) & (hits['layer_id']==6)]
#     fhits = hits[~hits['hit_id'].isin(nhits['hit_id'])]
#     fcells = cells[~cells['hit_id'].isin(fhits['hit_id'])]
#     ftruth = truth[~truth['hit_id'].isin(fhits['hit_id'])]
#     fparticles = particles[particles['particle_id'].isin(ftruth['particle_id'])]
    
#     name = f"{store_dir}remove_v8l6/event00000{ev}-"
    
#     fhits.to_csv(name+"hits.csv", index=False)
#     fcells.to_csv(name+"cells.csv", index=False)
#     ftruth.to_csv(name+"truth.csv", index=False)
#     fparticles.to_csv(name+"particles.csv", index=False)

In [9]:
# # select 10 events, remove v8l6 
# for evtid in evts:
#     hits, cells, particles, truth = trackml.dataset.load_event(f"{og_evts}event00000{str(evtid)}")
#     remove_v8l6(hits,cells,particles,truth,store_dir,evtid)

In [10]:
# v,_,_,_ = trackml.dataset.load_event(f"/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/remove_v8l6/{eventid}")

In [11]:
# # check inference
# import os
# preprocess()
# v[(v['volume_id']==8) & (v['layer_id']==6)]

# Shift only x coordinate of every hit

In [12]:
def shift_x(hits,cells,particles,truth,store_dir,ev,mis):
    
    # 1000 micron = 1 milimeter
    # one direction
    hits.loc[:, 'x'] = hits['x']+mis
    
    name = f"{store_dir}shift_x_all/{mis}"
    Path(name).mkdir(parents=True, exist_ok=True)
    name = name + f'/event00000{ev}-'
    
    hits.to_csv(name+"hits.csv", index=False)
    cells.to_csv(name+"cells.csv", index=False)
    truth.to_csv(name+"truth.csv", index=False)
    particles.to_csv(name+"particles.csv", index=False)

In [13]:
for mis in tqdm(misl):
    print(f"------ {mis} ------")
    for evtid in tq.tqdm(evts):
        hits, cells, particles, truth = trackml.dataset.load_event(f"{event_path}event00000{str(evtid)}")
        shift_x(hits,cells,particles,truth,store_dir,evtid,mis)

  0%|          | 0/14 [00:00<?, ?it/s]

------ 0 ------


  7%|▋         | 1/14 [11:06<2:24:21, 666.24s/it]


------ 0.0025 ------


 14%|█▍        | 2/14 [15:50<1:50:21, 551.76s/it]

...

 86%|████████▌ | 12/14 [1:02:51<09:37, 288.86s/it]


------ 0.8 ------


 93%|█████████▎| 13/14 [1:07:31<04:46, 286.14s/it]


------ 1 ------


100%|██████████| 14/14 [1:12:05<00:00, 308.95s/it]

In [14]:
# oh,oc,op,ot = trackml.dataset.load_event(f"{og_evts}{eventid}")
# oh

In [15]:
# h,c,p,t = trackml.dataset.load_event(f"/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/shift_x/1/{eventid}")
# h

# Shift only x coordinate of Volume 8 all layers

In [16]:
# def shift_x_v8(hits,cells,particles,truth,store_dir,ev,mis):
    
#     # 1000 micron = 1 milimeter
#     #one direction
#     hits.loc[(hits['volume_id']==8), 'x'] = hits[hits['volume_id']==8]['x']+mis
# #     hits.loc[(hits['volume_id']==8) & (hits['x']<0), 'x'] = hits[hits['volume_id']==8]['x']-mis
# #     hits.loc[(hits['volume_id']==8) & (hits['x']>0), 'x'] = hits[hits['volume_id']==8]['x']+mis
    
#     name = f"{store_dir}shift_x_v8/{mis}/event00000{ev}-"
    
#     hits.to_csv(name+"hits.csv", index=False)
#     cells.to_csv(name+"cells.csv", index=False)
#     truth.to_csv(name+"truth.csv", index=False)
#     particles.to_csv(name+"particles.csv", index=False)

In [17]:
# for mis in misl:
#     for evtid in evts:
#         hits, cells, particles, truth = trackml.dataset.load_event(f"{og_evts}event00000{str(evtid)}")
#         shift_x_v8(hits,cells,particles,truth,store_dir,evtid,mis)

In [18]:
# oh,oc,op,ot = trackml.dataset.load_event(f"{og_evts}{eventid}")
# oh[oh['volume_id'] == 8]

,hit_id,x,y,z,volume_id,layer_id,module_id
15213,15214,-32.226700,-4.586000,-474.190002,8,2,1
15214,15215,-32.626598,-3.442120,-454.467987,8,2,1
15215,15216,-30.083700,-14.257500,-461.166992,8,2,2
15216,15217,-18.472500,-26.547300,-477.484985,8,2,3
15217,15218,-20.716000,-25.466200,-468.579010,8,2,3
...,...,...,...,...,...,...,...
41449,41450,-168.143997,36.557201,483.890015,8,8,1090
41450,41451,-170.589996,25.515100,423.203003,8,8,1091
41451,41452,-170.440002,17.709600,419.201996,8,8,1091
41452,41453,-170.417999,16.547100,430.428009,8,8,1091


In [19]:
# h,c,p,t = trackml.dataset.load_event(f"/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/shift_x_v8/1/{eventid}")
# h[h['volume_id'] == 8]

,hit_id,x,y,z,volume_id,layer_id,module_id
15213,15214,-31.226700,-4.586000,-474.190002,8,2,1
15214,15215,-31.626598,-3.442120,-454.467987,8,2,1
15215,15216,-29.083700,-14.257500,-461.166992,8,2,2
15216,15217,-17.472500,-26.547300,-477.484985,8,2,3
15217,15218,-19.716000,-25.466200,-468.579010,8,2,3
...,...,...,...,...,...,...,...
41449,41450,-167.143997,36.557201,483.890015,8,8,1090
41450,41451,-169.589996,25.515100,423.203003,8,8,1091
41451,41452,-169.440002,17.709600,419.201996,8,8,1091
41452,41453,-169.417999,16.547100,430.428009,8,8,1091


# Shift x and y coordinates of all layers in volume 8

In [20]:
# def shift_xy_v8(hits,cells,particles,truth,store_dir,ev,mis):
    
#     # 1000 micron = 1 milimeter
#     # two directions
# #     hits.loc[(hits['volume_id']==8) & (hits['x']<0), 'x'] = hits[hits['volume_id']==8]['x']-mis
# #     hits.loc[(hits['volume_id']==8) & (hits['x']>0), 'x'] = hits[hits['volume_id']==8]['x']+mis
# #     hits.loc[(hits['volume_id']==8) & (hits['y']<0), 'x'] = hits[hits['volume_id']==8]['y']-mis
# #     hits.loc[(hits['volume_id']==8) & (hits['y']>0), 'x'] = hits[hits['volume_id']==8]['y']+mis
#     # one direction
#     hits.loc[(hits['volume_id']==8), 'x'] = hits[hits['volume_id']==8]['x']+mis
#     hits.loc[(hits['volume_id']==8), 'y'] = hits[hits['volume_id']==8]['y']+mis
    
#     name = f"{store_dir}shift_xy_v8/{mis}/event00000{ev}-"
    
#     hits.to_csv(name+"hits.csv", index=False)
#     cells.to_csv(name+"cells.csv", index=False)
#     truth.to_csv(name+"truth.csv", index=False)
#     particles.to_csv(name+"particles.csv", index=False)

In [21]:
# for mis in misl:
#     for evtid in evts:
#         hits, cells, particles, truth = trackml.dataset.load_event(f"{og_evts}event00000{str(evtid)}")
#         shift_xy_v8(hits,cells,particles,truth,store_dir,evtid,mis)

In [22]:
# oh,oc,op,ot = trackml.dataset.load_event(f"{og_evts}{eventid}")
# oh[oh['volume_id'] == 8]

,hit_id,x,y,z,volume_id,layer_id,module_id
15213,15214,-32.226700,-4.586000,-474.190002,8,2,1
15214,15215,-32.626598,-3.442120,-454.467987,8,2,1
15215,15216,-30.083700,-14.257500,-461.166992,8,2,2
15216,15217,-18.472500,-26.547300,-477.484985,8,2,3
15217,15218,-20.716000,-25.466200,-468.579010,8,2,3
...,...,...,...,...,...,...,...
41449,41450,-168.143997,36.557201,483.890015,8,8,1090
41450,41451,-170.589996,25.515100,423.203003,8,8,1091
41451,41452,-170.440002,17.709600,419.201996,8,8,1091
41452,41453,-170.417999,16.547100,430.428009,8,8,1091


In [23]:
# h,c,p,t = trackml.dataset.load_event(f"/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/shift_xy_v8/1/{eventid}")
# h[h['volume_id'] == 8]

,hit_id,x,y,z,volume_id,layer_id,module_id
15213,15214,-31.226700,-3.586000,-474.190002,8,2,1
15214,15215,-31.626598,-2.442120,-454.467987,8,2,1
15215,15216,-29.083700,-13.257500,-461.166992,8,2,2
15216,15217,-17.472500,-25.547300,-477.484985,8,2,3
15217,15218,-19.716000,-24.466200,-468.579010,8,2,3
...,...,...,...,...,...,...,...
41449,41450,-167.143997,37.557201,483.890015,8,8,1090
41450,41451,-169.589996,26.515100,423.203003,8,8,1091
41451,41452,-169.440002,18.709600,419.201996,8,8,1091
41452,41453,-169.417999,17.547100,430.428009,8,8,1091


# Shift x coordinate of endcap volume 12

In [24]:
# def shift_x_v12(hits,cells,particles,truth,store_dir,ev,mis):
    
#     # 1000 micron = 1 milimeter
#     # two directions
# #     hits.loc[(hits['volume_id']==8) & (hits['x']<0), 'x'] = hits[hits['volume_id']==8]['x']-mis
# #     hits.loc[(hits['volume_id']==8) & (hits['x']>0), 'x'] = hits[hits['volume_id']==8]['x']+mis
# #     hits.loc[(hits['volume_id']==8) & (hits['y']<0), 'x'] = hits[hits['volume_id']==8]['y']-mis
# #     hits.loc[(hits['volume_id']==8) & (hits['y']>0), 'x'] = hits[hits['volume_id']==8]['y']+mis
#     # one direction
#     hits.loc[(hits['volume_id']==12), 'x'] = hits[hits['volume_id']==12]['x']+mis
#     hits.loc[(hits['volume_id']==12), 'y'] = hits[hits['volume_id']==12]['y']+mis
    
#     name = f"{store_dir}shift_x_v12/{mis}/event00000{ev}-"
    
#     hits.to_csv(name+"hits.csv", index=False)
#     cells.to_csv(name+"cells.csv", index=False)
#     truth.to_csv(name+"truth.csv", index=False)
#     particles.to_csv(name+"particles.csv", index=False)

In [25]:
# for mis in misl:
#     for evtid in evts:
#         hits, cells, particles, truth = trackml.dataset.load_event(f"{og_evts}event00000{str(evtid)}")
#         shift_x_v12(hits,cells,particles,truth,store_dir,evtid,mis)

In [26]:
# oh,oc,op,ot = trackml.dataset.load_event(f"{og_evts}{eventid}")
# oh[oh['volume_id'] == 12]

,hit_id,x,y,z,volume_id,layer_id,module_id
56290,56291,-258.243988,-13.638600,-2948.5,12,2,1
56291,56292,-243.076996,-5.743410,-2948.5,12,2,1
56292,56293,-386.362000,-42.857399,-2948.5,12,2,1
56293,56294,-346.434998,-6.033630,-2948.5,12,2,1
56294,56295,-334.622986,-23.095699,-2948.5,12,2,1
...,...,...,...,...,...,...,...
63435,63436,-524.093994,45.137199,-1221.5,12,12,169
63436,63437,-550.841003,48.028000,-1215.5,12,12,170
63437,63438,-683.362976,72.103302,-1215.5,12,12,170
63438,63439,-592.309021,21.067900,-1215.5,12,12,170


In [27]:
# h,c,p,t = trackml.dataset.load_event(f"/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/shift_x_v12/1/{eventid}")
# h[h['volume_id'] == 12]

In [6]:
def shift(hits,cells,particles,truth,store_dir,ev,mis,vol):
    
    hits.loc[(hits['volume_id']==vol), f'x'] = hits[hits['volume_id']==vol][f'x']+mis
    
    name = f"{store_dir}shift_x_{vol}/{mis}"
    Path(name).mkdir(parents=True, exist_ok=True)
    name = name + f'/event00000{ev}-'
    
    hits.to_csv(name+"hits.csv", index=False)
    cells.to_csv(name+"cells.csv", index=False)
    truth.to_csv(name+"truth.csv", index=False)
    particles.to_csv(name+"particles.csv", index=False)

In [7]:
vols = [7,8,9,12,13,14,16,17,18]
misl = [0.4,0.6,0.8,1]
evts = np.arange(1000,1050,1)

# Shift x of all volume by 0 to 1 mm

In [8]:
for mis in tqdm(misl):
    print(f"------ {mis} ------")
    for evtid in tq.tqdm(evts):
        for vol in vols:
            hits, cells, particles, truth = trackml.dataset.load_event(f"{event_path}event00000{str(evtid)}")
            shift(hits,cells,particles,truth,store_dir,evtid,mis,vol)

  0%|          | 0/4 [00:00<?, ?it/s]

------ 0.4 ------


 25%|██▌       | 1/4 [20:15<1:00:47, 1215.91s/it]


------ 0.6 ------


 50%|█████     | 2/4 [40:15<40:22, 1211.11s/it]  


------ 0.8 ------


 75%|███████▌  | 3/4 [1:01:14<20:25, 1225.26s/it]


------ 1 ------


100%|██████████| 4/4 [1:21:23<00:00, 1220.81s/it]